<a href="https://colab.research.google.com/github/shmuhammadd/semantic_relatedness/blob/main/Simple_English_Baseline_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Package Imports

In [167]:
import pandas as pd
import re
from scipy.stats import spearmanr, pearsonr, linregress
import torch
import torch.nn.functional as F
from sklearn.preprocessing import StandardScaler
from transformers import AutoTokenizer, AutoModel
import Levenshtein
import matplotlib.pyplot as plt
plt.style.use('ggplot')

# Data Import / Format / Export

Functions for importing, formatting, and exporting data

In [168]:
# Load data from csv, format into proper split
def load_data(filepath):
    data = pd.read_csv(filepath)
    data['Split_Text'] = data['Text'].apply(lambda x: x.split("\n"))
    data['Pred_Score'] = 0.0
    return data

# Preprocessing

In [169]:
def jaccard_similarity(s1, s2):
    set1, set2 = set(s1), set(s2)
    return len(set1.intersection(set2)) / len(set1.union(set2))

In [170]:
def word_overlap(s1, s2):
    set1, set2 = set(s1), set(s2)
    return len(set1.intersection(set2)) / len(set1)

In [171]:
def dice_score(s1,s2):
  s1 = s1.lower()
  s1_split = re.findall(r"\w+|[^\w\s]", s1, re.UNICODE)

  s2 = s2.lower()
  s2_split = re.findall(r"\w+|[^\w\s]", s2, re.UNICODE)

  dice_coef = len(set(s1_split).intersection(set(s2_split))) / (len(set(s1_split)) + len(set(s2_split)))
  return round(dice_coef, 2)

In [172]:
# Additional features added to RoBERTa embeddings
def compute_custom_metrics(row):
    metrics = {}
    cosine_sim = F.cosine_similarity(row["Embedding1"].unsqueeze(0), row["Embedding2"].unsqueeze(0))
    metrics["Cosine_Similarity"] = cosine_sim.item()

    set1 = set(row["Sentence1"].split())
    set2 = set(row["Sentence2"].split())
    jaccard_sim = len(set1.intersection(set2)) / len(set1.union(set2)) if len(set1.union(set2)) > 0 else 0
    metrics["Jaccard_Similarity"] = jaccard_sim

    metrics["Length_Diff"] = abs(len(row["Sentence1"].split()) - len(row["Sentence2"].split()))

    metrics['Levenshtein_Distance'] = Levenshtein.distance(row['Sentence1'], row['Sentence2'])

    word_overlap_score = word_overlap(row["Sentence1"].split(), row["Sentence2"].split())
    metrics['Word_Overlap'] = word_overlap_score
    
    dice = dice_score(row["Sentence1"], row["Sentence2"])
    metrics['Dice_Score'] = dice

    return metrics

In [173]:
tokenizer = AutoTokenizer.from_pretrained("xlm-roberta-base")
model = AutoModel.from_pretrained("xlm-roberta-base")

In [174]:
# Needed batch sizes due to memory issues
def get_roberta_embeddings(sentences, batch_size=32):
    embeddings_list = []
    for i in range(0, len(sentences), batch_size):
        batch_sentences = sentences[i:i+batch_size]
        inputs = tokenizer(batch_sentences, padding=True, truncation=True, return_tensors="pt")
        with torch.no_grad():
            outputs = model(**inputs)
            embeddings = outputs.last_hidden_state[:, 0, :]
            embeddings_list.append(embeddings)
    return torch.cat(embeddings_list, dim=0)

In [175]:
def preprocess_with_roberta(data, batch_size=32):
    # Split into two sentences
    data[['Sentence1', 'Sentence2']] = pd.DataFrame(data['Split_Text'].tolist(), index=data.index)
    
    # Lowercase sentences, strip whitespace
    data["Sentence1"] = data["Sentence1"].str.lower().str.strip()
    data["Sentence2"] = data["Sentence2"].str.lower().str.strip()

    # Generate RoBERTa embeddings in batches (keeping everything as tensors)
    embeddings1 = get_roberta_embeddings(data["Sentence1"].tolist(), batch_size)
    embeddings2 = get_roberta_embeddings(data["Sentence2"].tolist(), batch_size)
    
    # Save embeddings for custom metrics
    data["Embedding1"] = list(embeddings1)
    data["Embedding2"] = list(embeddings2)

    # Compute custom metrics for each row
    metrics = data.apply(compute_custom_metrics, axis=1, result_type="expand")

    # Convert metrics to tensor
    metrics_tensor = torch.tensor(metrics.values, dtype=torch.float32)

    # Standardize custom metrics
    scaler = StandardScaler()
    standardized_metrics = scaler.fit_transform(metrics_tensor.numpy())
    standardized_metrics_tensor = torch.tensor(standardized_metrics, dtype=torch.float32)

    # Combine embeddings and metrics
    features = torch.cat([
        embeddings1,
        embeddings2,
        standardized_metrics_tensor
    ], dim=1)

    # Returned processed features as tensors
    return features, data

# Tools

In [176]:
def calculate_metrics(preds, scores):
    pearson_corr, _ = pearsonr(scores, preds)
    spearman_corr, _ = spearmanr(scores, preds)
    _, _, r, _, _ = linregress(scores, preds) # probably a better way of doing this, fix later
    r2 = r**2
    mse = ((scores - preds)**2).mean() # Scikit's mean_squared_error complained about being deprecated, so this is my temp fix
    return (pearson_corr, spearman_corr, r2, mse)

In [177]:
# Make sure these match the metrics above
def display_metrics(metrics, title="Metrics:"):
    print(title)
    print("Pearson Corr:", metrics[0])
    print("Spearman Corr:", metrics[1])
    print("R^2:", metrics[2])
    print("MSE:", metrics[3])

# Load data

In [ ]:
lang = 'eng'
#lang = 'esp' # This has no labeled testing data???
#lang = 'hau'
#lang = 'tel'
#lang = 'arq'
#lang = 'amh'
#lang = 'ary'
#lang = 'kin'
#lang = 'mar'


In [179]:
train_data = load_data(f"./Semantic_Relatedness_SemEval2024/Track A/{lang}/{lang}_train.csv")
train_data.head()

,PairID,Text,Score,Split_Text,Pred_Score
0,ARY-train-0000,تنفرد بنشرها “كود”: سلسلة “كيفاش السلاطين ديال...,0.38,[تنفرد بنشرها “كود”: سلسلة “كيفاش السلاطين ديا...,0.0
1,ARY-train-0001,الوضعية الوبائية لفيروس كورونا فهاد 24 ساعة: 2...,0.50,[الوضعية الوبائية لفيروس كورونا فهاد 24 ساعة: ...,0.0
2,ARY-train-0002,هاباش سالا ملف الاتجار في العفو الملكي بسجن ال...,0.53,[هاباش سالا ملف الاتجار في العفو الملكي بسجن ا...,0.0
3,ARY-train-0003,تنفرد بنشرها “كود”: سلسلة “كيفاش السلاطين ديال...,0.25,[تنفرد بنشرها “كود”: سلسلة “كيفاش السلاطين ديا...,0.0
4,ARY-train-0004,في أقل من سيمانة.. 5224 قضية تعرضات على القضاء...,0.56,[في أقل من سيمانة.. 5224 قضية تعرضات على القضا...,0.0


In [180]:
test_data = load_data(f"./Semantic_Relatedness_SemEval2024/Track A/{lang}/{lang}_test_with_labels.csv")
test_data.head()

,PairID,Text,Score,Split_Text,Pred_Score
0,ARY-test-0000,المحاكمات “عن بعد”.. ها شحال من جلسة دازت غير ...,0.66,[المحاكمات “عن بعد”.. ها شحال من جلسة دازت غير...,0.0
1,ARY-test-0001,أش واقع اليوم : حصيلة هاد النهار من الجرائم و ...,0.75,[أش واقع اليوم : حصيلة هاد النهار من الجرائم و...,0.0
2,ARY-test-0002,الأرصاد الجوية تحذر من رياح قوية السرعة ديالها...,0.75,[الأرصاد الجوية تحذر من رياح قوية السرعة دياله...,0.0
3,ARY-test-0003,تنفرد بنشرها “كود” سلسلة “كيفاش السلاطين ديال ...,0.36,[تنفرد بنشرها “كود” سلسلة “كيفاش السلاطين ديال...,0.0
4,ARY-test-0004,الرحيل ـ دمعة مسافرة ـ الحلقة السادسة والعشرون...,0.78,[الرحيل ـ دمعة مسافرة ـ الحلقة السادسة والعشرو...,0.0


In [181]:
train_features, train_data = preprocess_with_roberta(train_data)
print(train_data.shape)
train_data.head()

(924, 9)


,PairID,Text,Score,Split_Text,Pred_Score,Sentence1,Sentence2,Embedding1,Embedding2
0,ARY-train-0000,تنفرد بنشرها “كود”: سلسلة “كيفاش السلاطين ديال...,0.38,[تنفرد بنشرها “كود”: سلسلة “كيفاش السلاطين ديا...,0.0,تنفرد بنشرها “كود”: سلسلة “كيفاش السلاطين ديال...,تنفرد بنشرها “كود” سلسلة “كيفاش السلاطين ديال ...,"[tensor(0.1129), tensor(0.0950), tensor(0.0852...","[tensor(0.1217), tensor(0.1187), tensor(0.0862..."
1,ARY-train-0001,الوضعية الوبائية لفيروس كورونا فهاد 24 ساعة: 2...,0.50,[الوضعية الوبائية لفيروس كورونا فهاد 24 ساعة: ...,0.0,الوضعية الوبائية لفيروس كورونا فهاد 24 ساعة: 2...,حصيلة كورونا فهاد 24 ساعة: 1143 مغربي ومغربية ...,"[tensor(0.1455), tensor(0.1473), tensor(0.0857...","[tensor(0.1408), tensor(0.1307), tensor(0.0666..."
2,ARY-train-0002,هاباش سالا ملف الاتجار في العفو الملكي بسجن ال...,0.53,[هاباش سالا ملف الاتجار في العفو الملكي بسجن ا...,0.0,هاباش سالا ملف الاتجار في العفو الملكي بسجن ال...,جورنالات بلاديفضيحة الاتجار في العفو الملكي تج...,"[tensor(0.0767), tensor(0.0970), tensor(0.0708...","[tensor(0.0850), tensor(0.0961), tensor(0.0688..."
3,ARY-train-0003,تنفرد بنشرها “كود”: سلسلة “كيفاش السلاطين ديال...,0.25,[تنفرد بنشرها “كود”: سلسلة “كيفاش السلاطين ديا...,0.0,تنفرد بنشرها “كود”: سلسلة “كيفاش السلاطين ديال...,سلسلة تنفرد بنشرها “كود”:”كيفاش السلاطين ديال ...,"[tensor(0.1067), tensor(0.1069), tensor(0.0849...","[tensor(0.1294), tensor(0.1061), tensor(0.0952..."
4,ARY-train-0004,في أقل من سيمانة.. 5224 قضية تعرضات على القضاء...,0.56,[في أقل من سيمانة.. 5224 قضية تعرضات على القضا...,0.0,في أقل من سيمانة.. 5224 قضية تعرضات على القضاء...,المحاكمات “عن بعد”.. ها شحال من جلسة دازت غير ...,"[tensor(0.1212), tensor(0.0781), tensor(0.0859...","[tensor(0.1198), tensor(0.0815), tensor(0.0830..."


In [182]:
print(train_features.shape)
print(train_features)
torch.save(train_features, f"train_features_{lang}.pt")

torch.Size([924, 1542])
tensor([[ 0.1129,  0.0950,  0.0852,  ..., -0.0802,  0.7262,  0.7712],
        [ 0.1455,  0.1473,  0.0857,  ...,  0.8941, -1.3406, -0.6001],
        [ 0.0767,  0.0970,  0.0708,  ...,  0.7271,  0.2669, -0.9741],
        ...,
        [ 0.1066,  0.1096,  0.0656,  ..., -1.3329,  0.9558,  0.8959],
        [ 0.1245,  0.1251,  0.0687,  ..., -0.4143,  1.4151,  1.1452],
        [ 0.1493,  0.0757,  0.0899,  ..., -1.0267,  1.0132,  1.2699]])


In [183]:
train_labels = torch.tensor(train_data['Score'], dtype=torch.float32)
print(train_labels.shape)
print(train_labels)
torch.save(train_labels, f"train_labels_{lang}.pt")

torch.Size([924])
tensor([0.3800, 0.5000, 0.5300, 0.2500, 0.5600, 0.4200, 0.3900, 0.6600, 0.3100,
        0.7500, 0.5000, 0.8900, 0.2900, 0.3800, 0.6600, 0.2200, 0.5300, 0.7200,
        0.7800, 0.3900, 0.7100, 0.2800, 0.6100, 0.5700, 0.4100, 0.4700, 0.1700,
        0.2500, 0.7100, 0.5400, 0.3900, 0.1600, 0.5000, 0.0400, 0.4600, 0.3300,
        0.3800, 0.6100, 0.5000, 0.4600, 0.7100, 0.3200, 0.1800, 0.8600, 0.1700,
        0.1600, 0.4100, 0.5400, 0.7900, 0.6400, 0.3800, 0.5600, 0.8200, 0.4300,
        0.6900, 0.7000, 0.7500, 0.6100, 0.6400, 0.8400, 0.6200, 0.5300, 0.7900,
        0.5700, 0.3100, 0.4100, 0.5700, 0.5800, 0.3400, 0.6600, 0.3200, 0.4400,
        0.6200, 0.7200, 0.4100, 0.7500, 0.7800, 0.6800, 0.5700, 0.7500, 0.4200,
        0.1700, 0.3400, 0.7900, 0.3800, 0.2500, 0.5800, 0.7500, 0.3400, 0.3900,
        0.1600, 0.7800, 0.1700, 0.7500, 0.5000, 0.5700, 0.5000, 0.4700, 0.5300,
        0.5700, 0.7500, 0.7500, 0.7500, 0.8400, 0.4400, 0.6600, 0.5600, 0.1100,
        0.7900, 0.3900

In [184]:
test_features, test_data = preprocess_with_roberta(test_data)
print(test_data.shape)
test_data.head()

(426, 9)


,PairID,Text,Score,Split_Text,Pred_Score,Sentence1,Sentence2,Embedding1,Embedding2
0,ARY-test-0000,المحاكمات “عن بعد”.. ها شحال من جلسة دازت غير ...,0.66,[المحاكمات “عن بعد”.. ها شحال من جلسة دازت غير...,0.0,المحاكمات “عن بعد”.. ها شحال من جلسة دازت غير ...,المحاكمات عن بعد.. اكثر من 98 ألف محابسي استاف...,"[tensor(0.1198), tensor(0.0815), tensor(0.0830...","[tensor(0.1902), tensor(0.1000), tensor(0.0980..."
1,ARY-test-0001,أش واقع اليوم : حصيلة هاد النهار من الجرائم و ...,0.75,[أش واقع اليوم : حصيلة هاد النهار من الجرائم و...,0.0,أش واقع اليوم : حصيلة هاد النهار من الجرائم و ...,أش واقع اليوم : حصيلة هاد النهار من الجرائم و ...,"[tensor(0.1316), tensor(0.1006), tensor(0.1035...","[tensor(0.1158), tensor(0.0956), tensor(0.0940..."
2,ARY-test-0002,الأرصاد الجوية تحذر من رياح قوية السرعة ديالها...,0.75,[الأرصاد الجوية تحذر من رياح قوية السرعة دياله...,0.0,الأرصاد الجوية تحذر من رياح قوية السرعة ديالها...,الأرصاد الجوية كتحذر من ريح قوية غادي توصل حتى...,"[tensor(0.0952), tensor(0.0928), tensor(0.0698...","[tensor(0.0782), tensor(0.0959), tensor(0.0740..."
3,ARY-test-0003,تنفرد بنشرها “كود” سلسلة “كيفاش السلاطين ديال ...,0.36,[تنفرد بنشرها “كود” سلسلة “كيفاش السلاطين ديال...,0.0,تنفرد بنشرها “كود” سلسلة “كيفاش السلاطين ديال ...,سلسلة تنفرد بنشرها “كود” “كيفاش السلاطين ديال ...,"[tensor(0.0927), tensor(0.0857), tensor(0.0865...","[tensor(0.0924), tensor(0.1043), tensor(0.0800..."
4,ARY-test-0004,الرحيل ـ دمعة مسافرة ـ الحلقة السادسة والعشرون...,0.78,[الرحيل ـ دمعة مسافرة ـ الحلقة السادسة والعشرو...,0.0,الرحيل ـ دمعة مسافرة ـ الحلقة السادسة والعشرون,الرحيل ـ دمعة مسافرة ـ الحلقة الرابعة والعشرون,"[tensor(0.0949), tensor(0.0952), tensor(0.0537...","[tensor(0.1031), tensor(0.1083), tensor(0.0551..."


In [185]:
print(test_features.shape)
print(test_features)
torch.save(test_features, f"test_features_{lang}.pt")

torch.Size([426, 1542])
tensor([[ 0.1198,  0.0815,  0.0830,  ..., -0.3684, -0.3205,  0.2585],
        [ 0.1316,  0.1006,  0.1035,  ..., -0.1506,  2.6187,  1.6057],
        [ 0.0952,  0.0928,  0.0698,  ..., -1.2122,  0.9230,  0.8708],
        ...,
        [ 0.0842,  0.1105,  0.0786,  ...,  2.1087, -1.4752, -1.8236],
        [ 0.1491,  0.1447,  0.0986,  ...,  1.3737, -1.4354, -1.2112],
        [ 0.0844,  0.0970,  0.0714,  ...,  1.7004, -0.9318, -1.5787]])


In [186]:
test_labels = torch.tensor(test_data['Score'], dtype=torch.float32)
print(test_labels.shape)
print(test_labels)
torch.save(test_labels, f"test_labels_{lang}.pt")

torch.Size([426])
tensor([0.6600, 0.7500, 0.7500, 0.3600, 0.7800, 1.0000, 0.2500, 0.4000, 0.4700,
        0.4600, 0.2100, 0.1200, 0.7200, 0.4400, 0.3800, 0.5700, 0.2500, 0.5000,
        0.6600, 0.7500, 0.3200, 0.5700, 0.5000, 0.5400, 0.3800, 0.9300, 0.6200,
        0.6100, 0.5700, 0.2800, 0.5600, 0.3900, 0.3400, 0.6100, 0.5900, 0.4700,
        0.4400, 0.4400, 0.7500, 0.5900, 0.5900, 0.1600, 0.4100, 0.4400, 0.8900,
        0.6800, 0.2900, 0.6200, 0.4600, 0.2500, 0.6800, 0.5000, 0.4400, 0.5400,
        0.7500, 0.2800, 0.6600, 0.5700, 0.6800, 0.1200, 0.2200, 0.1900, 0.6200,
        0.1900, 0.3800, 0.8600, 0.7800, 0.5300, 0.6200, 0.5300, 0.4300, 0.5600,
        0.8200, 0.5300, 0.2200, 0.6700, 0.6400, 0.2100, 0.4300, 0.3100, 0.2800,
        0.6600, 0.6200, 0.6100, 0.2200, 0.4100, 0.7200, 0.3100, 0.7900, 0.1600,
        0.7900, 0.2900, 0.4100, 0.5900, 0.4200, 0.6600, 0.5000, 0.5300, 0.5300,
        0.5400, 0.6900, 0.6100, 0.4600, 0.2900, 0.1900, 0.5900, 0.2100, 0.1600,
        0.2500, 0.6600